In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
%matplotlib inline

import os, math
import keras.backend as K
from numpy import random
from __future__ import division
from sklearn import dummy, metrics, cross_validation, ensemble
from keras.layers import Input, Embedding, Flatten, Dropout, Conv2D, merge, normalization, MaxPooling1D,Dense, Dot, Concatenate, Merge, Conv1D, Add,add
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.models import Model
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau

import keras
import deepdish as dd
from read_activations import *

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
a_songid = dd.io.load('/home/jvidyala/model_sample_data/a_songid.h5')
b_songid = dd.io.load('/home/jvidyala/model_sample_data/b_songid.h5')
a_userid = dd.io.load('/home/jvidyala/model_sample_data/a_userid.h5')
b_userid = dd.io.load('/home/jvidyala/model_sample_data/b_userid.h5')
a_y = dd.io.load('/home/jvidyala/model_sample_data/a_y.h5')
b_y = dd.io.load('/home/jvidyala/model_sample_data/b_y.h5')

In [7]:
n_users = 1019318
n_songs = 384546

Model 1

Basic MF 

In [8]:
song_input = Input(shape=[1])
song_embedding = Flatten()(Embedding(n_songs+1, 16)(song_input))
song_vec = (song_embedding)

user_input = Input(shape=[1])
user_embedding = Flatten()(Embedding(n_users+1, 16)(user_input))
user_vec = (user_embedding)
    
input_vecs = merge([song_vec,user_vec],mode='dot')    
x = Dense(128, activation='relu')(input_vecs)
x = Dense(128, activation='relu')(input_vecs)

y = Dense(1)(x)
model = Model(inputs=[song_input,user_input],outputs=y)
model.compile(loss='mse',optimizer='adam')

In [9]:
history = model.fit([a_songid,a_userid], a_y,
                   epochs = 10,
                   validation_data=([b_songid, b_userid], b_y),
                   batch_size=64, shuffle=True)

Train on 47992 samples, validate on 2020 samples
Epoch 1/10
47992/47992 [==============================] - 41s 856us/step - loss: 105.4570 - val_loss: 46.6448
Epoch 2/10
47992/47992 [==============================] - 21s 432us/step - loss: 102.0435 - val_loss: 47.5735
Epoch 3/10
47992/47992 [==============================] - 21s 432us/step - loss: 94.4999 - val_loss: 49.2295
Epoch 4/10
47992/47992 [==============================] - 21s 431us/step - loss: 87.5193 - val_loss: 50.5565
Epoch 5/10
47992/47992 [==============================] - 21s 432us/step - loss: 82.2357 - val_loss: 52.4793
Epoch 6/10
47992/47992 [==============================] - 21s 431us/step - loss: 78.6655 - val_loss: 52.5632
Epoch 7/10
47992/47992 [==============================] - 21s 433us/step - loss: 75.7201 - val_loss: 53.0329
Epoch 8/10
47992/47992 [==============================] - 21s 433us/step - loss: 72.9290 - val_loss: 53.3300
Epoch 9/10
47992/47992 [==============================] - 21s 434us/step - lo

Model 2

Bias added

In [12]:
song_input = Input(shape=[1])
song_embedding = Flatten()(Embedding(n_songs+1, 16)(song_input))
song_vec = Dropout(0.2)(song_embedding)


user_input = Input(shape=[1])
user_embedding = Flatten()(Embedding(n_users+1, 16)(user_input))
user_vec = Dropout(0.2)(user_embedding)

song_bias = Embedding(input_dim=n_songs,output_dim=1,input_length=1)(song_input)
user_bias = Embedding(input_dim=n_users,output_dim=1,input_length=1)(user_input)

input_vecs = Concatenate()([song_vec,user_vec]) 
x = add([input_vecs,song_bias,user_bias])
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(input_vecs)

y = Dense(1)(x)
model = Model(inputs=[song_input,user_input],outputs=y)
model.compile(loss='mse',optimizer='adam')

In [13]:
history = model.fit([a_songid,a_userid], a_y,
                   epochs = 10,
                   validation_data=([b_songid, b_userid], b_y),
                   batch_size=64, shuffle=True)

Train on 47992 samples, validate on 2020 samples
Epoch 1/10
47992/47992 [==============================] - 21s 446us/step - loss: 102.9100 - val_loss: 40.5931
Epoch 2/10
47992/47992 [==============================] - 21s 440us/step - loss: 98.4402 - val_loss: 38.1002
Epoch 3/10
47992/47992 [==============================] - 21s 439us/step - loss: 93.7318 - val_loss: 39.3461
Epoch 4/10
47992/47992 [==============================] - 21s 437us/step - loss: 89.4407 - val_loss: 41.3951
Epoch 5/10
47992/47992 [==============================] - 21s 438us/step - loss: 85.2411 - val_loss: 45.8071
Epoch 6/10
47992/47992 [==============================] - 21s 439us/step - loss: 82.3194 - val_loss: 44.4275
Epoch 7/10
47992/47992 [==============================] - 21s 438us/step - loss: 79.5438 - val_loss: 48.4840
Epoch 8/10
47992/47992 [==============================] - 21s 439us/step - loss: 77.3144 - val_loss: 50.0199
Epoch 9/10
47992/47992 [==============================] - 21s 440us/step - los

Model 3

Deep MF

In [15]:
user_input = Input(shape=[1])
user_embedding = Flatten()(Embedding(input_dim=n_users+1, output_dim=16)(user_input))
user_vec = Dense(16)(user_embedding)
user_vec = Dense(16)(user_vec)

song_input = Input(shape=[1])
song_embedding = Flatten()(Embedding(input_dim=n_songs+1, output_dim=16)(song_input))
song_vec = Dense(16)(song_embedding)
song_vec = Dense(16)(song_vec)

prediction = merge([user_vec,song_vec],mode='dot')
prediction = Dense(16)(prediction)
prediction = Dense(1)(prediction)

model = Model(inputs=[user_input,song_input],outputs=prediction)
model.compile(loss='mse',optimizer='adam')

In [16]:
history = model.fit([a_songid,a_userid], a_y,
                   epochs = 10,
                   validation_data=([b_songid, b_userid], b_y),
                   batch_size=64, shuffle=True)

Train on 47992 samples, validate on 2020 samples
Epoch 1/10
47992/47992 [==============================] - 22s 461us/step - loss: 104.3873 - val_loss: 44.5188
Epoch 2/10
47992/47992 [==============================] - 22s 449us/step - loss: 100.4317 - val_loss: 44.7838
Epoch 3/10
47992/47992 [==============================] - 22s 450us/step - loss: 95.1700 - val_loss: 48.0210
Epoch 4/10
47992/47992 [==============================] - 22s 449us/step - loss: 90.8325 - val_loss: 53.0685
Epoch 5/10
47992/47992 [==============================] - 22s 451us/step - loss: 86.7563 - val_loss: 57.4254
Epoch 6/10
47992/47992 [==============================] - 22s 452us/step - loss: 81.4962 - val_loss: 65.2434
Epoch 7/10
47992/47992 [==============================] - 22s 451us/step - loss: 73.9924 - val_loss: 78.2448
Epoch 8/10
47992/47992 [==============================] - 22s 452us/step - loss: 62.3539 - val_loss: 95.4498
Epoch 9/10
47992/47992 [==============================] - 22s 453us/step - lo